In [4]:
import numpy as np
import sys
import os, subprocess

In [5]:
def get_clock(Fs):

    '''
    clk_out = 100 MHz * (M/D) / 2

    M must be between 2 and 256
    D must be between 1 and 256

    Each chip samples all 32 channels + 3 auxiliary channels (temperature, etc, etc)

    A Complete SPI cycle takes 80 clock cycles
    for each channel + 3 auxilary = 80*(32 + 3) = 2800 clock cycles
        
    per channel sampling rate (Fs) is 2800 times slower than the clock
    
    clk_out (MHz) = 2.8 Fs (kHz)
    '''
    
    X = 2.8*(Fs)/50  # M = X*D
    
    epsilon = 1e-10
    
    output = np.array([])
    
    for m in np.arange(2,257):
        
        d = m/X
            
        # use an epsilon to account for float rounding errors
        mod_d = d % 1
        if mod_d <= epsilon and mod_d >= 0:
            # this satisfies an over estimated value due to float rounding
            if d >= 1 and d <= 256:
                if len(output) == 0:
                    output = np.array([int(m), int(np.floor(d))])
                else:
                    output = np.vstack((output, np.array([int(m), int(np.floor(d))])))
            
        elif mod_d < 1 and mod_d >= 1 - epsilon:
            # this is for slight underestimated float rounding
            if d >= 1 and d <= 256:
                if len(output) == 0:
                    output = np.array([int(m), int(np.ceil(d))])
                else:
                    output = np.vstack((output, np.array([int(m), int(np.ceil(d))])))
        
    return output, X

In [8]:
output, X = get_clock(20)
print(output)

[[ 28  25]
 [ 56  50]
 [ 84  75]
 [112 100]
 [140 125]
 [168 150]
 [196 175]
 [224 200]
 [252 225]]


In [57]:
X = 2.8*(2.5)/50  # M = X*D
d = 21/X
print(d)

print(d%1)

149.99999999999997
0.9999999999999716


# Adding FPGA values

1. Open rhd2000evalboard.cpp
2. Calculate the M and D values for a desired sampling frequency using get_clock()
3. Add a case for your sampling frequency to setSampleRate with the M and D values that you just calculated.
4. Add a case for your sampling frequency to getSampleRate so the proper Fs is provided

# Add the SampleRateXXXHz Value
1. open rhd2000evalboard.h
2. look in the enum AmplifierSampleRate section and add your SampleRate value

# Modifying the GUI

1. open mainwindow.cpp
2. Add your option to the sampleRateComboBox
3. Whenever the index value of this combo box is changed it will trigger sampleRateComboBox, so you will need to add a case for your index

# Making Static Qt version for Standalone Apps

1. get latest open source https://download.qt.io/archive/qt/5.10/5.10.1/single/
2. follow these instructions https://wohlsoft.ru/pgewiki/Building_static_Qt_5#Windows

# Compiling
- Make sure you have the Visual Studio Qt tool installed so you cna get the Qt VS Tools toolbar
- Add your Qt version to the Qt Options
- Make sure all your code is within a Qt project (can add source files to a solution)
- Make sure these Qt Modules are checked in the Qt Project Settings, Core, GUI, Multimedia, and Widgets
- add the windeployqt.exe path to your path variables (example: C:\Qt\5.11.0\msvc2017_64\bin\windeployqt)
    - Note if you have Anaconda3 installed for PyQt, you will have issues with this
- Add the appropriate okFrontPanel.dll file to your application directory

## Modifying GUI for Settings QLineEdit

- open mainwindow.cpp
- add the LineEdit to the MainWindow::createLayout()
    - make sure you add the name of this LineEdit to mainwindow.h
- modify the loadSettings() function and the saveSettings() function to set the text of the LineEdit
- Open globelconstants.h and change the version number